In [ ]:
# Import needed libraries
%reload_ext tensorboard

import datetime

# Data containers
import numpy as np
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Data Manipulation and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

# Deep_Learning
import tensorflow as tf
from tensorflow import keras

# This code snippet forces tensorflow to not automatically allocate all GPU ram which can be an issue in notebook environment
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
tf.debugging.disable_traceback_filtering()

### Define the Custom Layer for multiple activation functions per layer

In [ ]:
# Define custom Layer
class MultiActivationLayer(keras.layers.Layer):
    '''
    Multiple Activation Layer
    
    A neural network layer in which every node has a different activation function applied
    '''
    
    def __init__(self, out_features, activations, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features
        self.activations = activations
    
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
    
    @tf.function
    def call(self, inputs):
        z = tf.matmul(inputs, self.w) + self.b
        shape = tf.shape(z)[0]
        
        # Apply activation function to ouput features from nodes (columns) separately with different activation functions
        #, reshape to 2-D array and concatenate the results from each node in the same order
        nodes = [tf.reshape(self.activations[i%len(self.activations)](z[:,i]), (shape, 1)) for i in range(self.out_features)]
        z = tf.concat(nodes, 1)
        return z
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "out_features": self.out_features,
            "activations": self.activations,
        })
        return config

## Define method for creating the testing models with equivalent architectures

In [ ]:
# Define custom architecture using the new layer
def create_model(activations, option='multi', num_classes=1, dropout=False, dropout_rate=0.2, task='classification'):
    if option == 'uniform':
        if not dropout:
            if task == 'classification':
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[0], name='layers_dense_2'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[0], name='layers_dense_3'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[0], name='layers_dense_4'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[0], name='layers_dense_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                 return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[0], name='layers_dense_2'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[0], name='layers_dense_3'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[0], name='layers_dense_4'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[0], name='layers_dense_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
        else:
            if task == 'classification':
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[0], name='layers_dense_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[0], name='layers_dense_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[0], name='layers_dense_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[0], name='layers_dense_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[0], name='layers_dense_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[0], name='layers_dense_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[0], name='layers_dense_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[0], name='layers_dense_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
    elif option == 'multi':
        if not dropout:
            if task == 'classification':
                return keras.models.Sequential([
                    MultiActivationLayer(25, activations, name='layers_multi_1'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(20, activations, name='layers_multi_2'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(15, activations, name='layers_multi_3'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(10, activations, name='layers_multi_4'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(5, activations, name='layers_multi_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                 return keras.models.Sequential([
                    MultiActivationLayer(25, activations, name='layers_multi_1'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(20, activations, name='layers_multi_2'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(15, activations, name='layers_multi_3'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(10, activations, name='layers_multi_4'),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(5, activations, name='layers_multi_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
        else:
            if task == 'classification':
                return keras.models.Sequential([
                    MultiActivationLayer(25, activations, name='layers_multi_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(20, activations, name='layers_multi_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(15, activations, name='layers_multi_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(10, activations, name='layers_multi_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(5, activations, name='layers_multi_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                return keras.models.Sequential([
                    MultiActivationLayer(25, activations, name='layers_multi_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(20, activations, name='layers_multi_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(15, activations, name='layers_multi_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(10, activations, name='layers_multi_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    MultiActivationLayer(5, activations, name='layers_multi_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
    elif option == 'sequential':
        if len(activations) < 5: raise RuntimeError()
        if not dropout:
            if task == 'classification':
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[1], name='layers_dense_2'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[2], name='layers_dense_3'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[3], name='layers_dense_4'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[4], name='layers_dense_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                 return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[1], name='layers_dense_2'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[2], name='layers_dense_3'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[3], name='layers_dense_4'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[4], name='layers_dense_5'),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
        else: 
            if task == 'classification':
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[1], name='layers_dense_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[2], name='layers_dense_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[3], name='layers_dense_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[4], name='layers_dense_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(num_classes, activation='sigmoid' if num_classes < 2 else 'softmax', name='layers_dense')
                ])
            else:
                return keras.models.Sequential([
                    keras.layers.Dense(25, activation=activations[0], name='layers_dense_1'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(20, activation=activations[1], name='layers_dense_2'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(15, activation=activations[2], name='layers_dense_3'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(10, activation=activations[3], name='layers_dense_4'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(5, activation=activations[4], name='layers_dense_5'),
                    keras.layers.Dropout(dropout_rate),
                    keras.layers.BatchNormalization(),
                    keras.layers.Dense(1, name='layers_dense')
                ])
    else:
        raise RuntimeError()

## Define method for training all testing models on the data

In [ ]:
# Define testing function for generating and running tests
def test(X, y, num_classes, task='classification', epochs=100, batch_size=32):
    
    if task == 'classification':
    
        # Create our model
        MANN = create_model(activations=[tf.nn.sigmoid, tf.nn.tanh, tf.nn.leaky_relu, tf.nn.elu, tf.nn.swish], dropout=False, num_classes=num_classes, task=task)
        MANN.compile(optimizer=keras.optimizers.legacy.RMSprop(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])

        # Create our model with dropout
        MANN_drop = create_model(activations=[tf.nn.sigmoid, tf.nn.tanh, tf.nn.leaky_relu, tf.nn.elu, tf.nn.swish], dropout=True, num_classes=num_classes, task=task)
        MANN_drop.compile(optimizer=keras.optimizers.legacy.RMSprop(),
                         loss=keras.losses.CategoricalCrossentropy(),
                         metrics=[keras.metrics.CategoricalAccuracy()])

        # Create uniform model for each activation function
        UANN1 = create_model(activations=[tf.nn.sigmoid], option='uniform', num_classes=num_classes, task=task)
        UANN1.compile(optimizer=keras.optimizers.legacy.RMSprop(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        UANN2 = create_model(activations=[tf.nn.tanh], option='uniform', num_classes=num_classes, task=task)
        UANN2.compile(optimizer=keras.optimizers.legacy.RMSprop(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        UANN3 = create_model(activations=[tf.nn.leaky_relu], option='uniform', num_classes=num_classes, task=task)
        UANN3.compile(optimizer=keras.optimizers.legacy.RMSprop(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        UANN4 = create_model(activations=[tf.nn.elu], option='uniform', num_classes=num_classes, task=task)
        UANN4.compile(optimizer=keras.optimizers.legacy.RMSprop(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        UANN5 = create_model(activations=[tf.nn.swish], option='uniform', num_classes=num_classes, task=task)
        UANN5.compile(optimizer=keras.optimizers.legacy.RMSprop(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        # Create sequential activation network
        SANN = create_model(activations=[tf.nn.swish, tf.nn.elu, tf.nn.leaky_relu, tf.nn.tanh, tf.nn.sigmoid], option='sequential', dropout=False, num_classes=num_classes, task=task)
        SANN.compile(optimizer=keras.optimizers.legacy.RMSprop(),
                     loss=keras.losses.CategoricalCrossentropy(),
                     metrics=[keras.metrics.CategoricalAccuracy()])
        
        # Split data into training and testing
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)

        train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
        test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

        train_dataset = train_dataset.shuffle(1000).batch(256)
        test_dataset = test_dataset.batch(256)
        
        # Fit the data to the models for a set number of epochs
        print("Training Multi Activation Neural Network...")
        MANN.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, verbose=0)
        print('')
        
        print("Training Multi Activation Neural Network w/ Dropout...")
        MANN_drop.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, verbose=0)
        print('Finished.')
        
        
        print("Training Sigmoid Uniform Activation Neural Network...")
        UANN1.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, verbose=0)
        print('Finished.')
        
        print("Training Tanh Uniform Activation Neural Network...")
        UANN2.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, verbose=0)
        print('Finished.')
        
        print("Training Leaky ReLU Uniform Activation Neural Network...")
        UANN3.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, verbose=0)
        print('Finished.')
        
        print("Training ELU Uniform Activation Neural Network...")
        UANN4.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, verbose=0)
        print('Finished.')
        
        print("Training Swish Uniform Activation Neural Network...")
        UANN5.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, verbose=0)
        print('Finished.')
        
        print("Training Sequential Activation Neural Network...")
        SANN.fit(x=train_dataset, batch_size=batch_size, epochs=epochs, validation_data=test_dataset, verbose=0)
        print('Finished.')
        
        # Create dictionaries to return for models and scores
        models = {
            'Multi': MANN,
            'Multi, Dropout': MANN_drop,
            'Uniform, Sigmoid': UANN1,
            'Uniform, Tanh': UANN2,
            'Uniform, Leaky ReLU': UANN3,
            'Uniform, ELU': UANN4,
            'Uniform, Swish': UANN5,
            'Sequential': SANN
        }
        
        return models

## Regression task

In [ ]:
# Load datasets and view
df = pd.read_csv('Stroke_Dataset_Normalized.csv', index_col=0)
df

In [ ]:
sns.pairplot(df, hue='classification', corner=True)
plt.show()

In [ ]:
df.describe()

In [ ]:
X = df[df.columns[:-1]].to_numpy(dtype=np.float32)

In [ ]:
y = df['classification'].to_numpy(dtype=np.float32)

In [ ]:
encoder = OneHotEncoder()
y = np.asarray(encoder.fit_transform(y.reshape((y.shape[0], 1))).todense())

In [ ]:
models = test(X, y, len(encoder.categories_[0]))